In [82]:
import torch
from torch import nn
from torch.autograd import Variable

In [83]:
import os
import os.path

import shutil

directory = "data/geoguessr_filtered_250/"

filenames = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if (os.path.isdir(f)):
        i = 0
        for image in os.listdir(f):
            filenames.append(f"{f}/{image}")
            i += 1

In [84]:
import geoLSTM

In [85]:
from shapely.geometry import Point, Polygon
from matplotlib import pyplot as plt
from math import sin, cos, sqrt, atan2, radians

# !pip install gmaps
import shapely
import pickle
import random
import numpy as np
import gmaps, os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from ipywidgets.embed import embed_minimal_html
import webbrowser
import argparse

from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, LSTM
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D, GlobalMaxPool3D
from tensorflow.keras.applications.resnet50 import ResNet50

In [86]:
filenames
fn1 = list(map(lambda x: "/".join(x.split("/")[2:]), filenames))
print(fn1)

['Aland/canvas_1629586212.jpg', 'Albania/canvas_1629319286.jpg', 'American Samoa/canvas_1629274653.jpg', 'Andorra/canvas_1629312152.jpg', 'Argentina/canvas_1629257860.jpg', 'Argentina/canvas_1629480118.jpg', 'Argentina/canvas_1629730075.jpg', 'Argentina/canvas_1629942298.jpg', 'Argentina/canvas_1630100848.jpg', 'Australia/canvas_1629257902.jpg', 'Australia/canvas_1629353514.jpg', 'Australia/canvas_1629448318.jpg', 'Australia/canvas_1629544181.jpg', 'Australia/canvas_1629683854.jpg', 'Australia/canvas_1629741251.jpg', 'Australia/canvas_1629828705.jpg', 'Australia/canvas_1629907075.jpg', 'Australia/canvas_1630001200.jpg', 'Australia/canvas_1630059063.jpg', 'Australia/canvas_1630120358.jpg', 'Australia/canvas_1630280723.jpg', 'Austria/canvas_1629258906.jpg', 'Austria/canvas_1629682638.jpg', 'Austria/canvas_1630091005.jpg', 'Bangladesh/canvas_1629273181.jpg', 'Belgium/canvas_1629260343.jpg', 'Belgium/canvas_1629969644.jpg', 'Bermuda/canvas_1629939285.jpg', 'Bhutan/canvas_1629527767.jpg', '

In [87]:
classes = {'American Samoa': 0, 'Palestine': 1, 'Sweden': 2, 'Guam': 3, 'Kenya': 4, 'Vietnam': 5, 'Chile': 6,
           'Ireland': 7, 'Armenia': 8, 'Canada': 9, 'Ghana': 10, 'Tunisia': 11, 'Bhutan': 12, 'Colombia': 13,
           'Philippines': 14, 'Belgium': 15, 'Greece': 16, 'Ukraine': 17, 'Tanzania': 18, 'Kyrgyzstan': 19,
           'Slovakia': 20, 'Eswatini': 21, 'Serbia': 22, 'Paraguay': 23, 'Indonesia': 24, 'Spain': 25,
           'Isle of Man': 26, 'China': 27, 'North Macedonia': 28, 'Curacao': 29, 'Ecuador': 30, 'Gibraltar': 31,
           'Italy': 32, 'Lebanon': 33, 'Pitcairn Islands': 34, 'Puerto Rico': 35, 'Bolivia': 36, 'Nigeria': 37,
           'Hong Kong': 38, 'South Sudan': 39, 'Uganda': 40, 'Iraq': 41, 'New Zealand': 42, 'Dominican Republic': 43,
           'Estonia': 44, 'Greenland': 45, 'Luxembourg': 46, 'Senegal': 47, 'Mongolia': 48, 'India': 49, 'Pakistan': 50,
           'United Arab Emirates': 51, 'Austria': 52, 'Singapore': 53, 'Nepal': 54, 'Cambodia': 55, 'Aland': 56,
           'Svalbard and Jan Mayen': 57, 'Lithuania': 58, 'Mozambique': 59, 'Denmark': 60, 'Israel': 61,
           'United Kingdom': 62, 'France': 63, 'Thailand': 64, 'Reunion': 65, 'Iceland': 66, 'Taiwan': 67, 'Jersey': 68,
           'Andorra': 69, 'Faroe Islands': 70, 'Mexico': 71, 'Turkey': 72, 'Romania': 73, 'Japan': 74, 'Myanmar': 75,
           'South Korea': 76, 'Laos': 77, 'Portugal': 78, 'Slovenia': 79, 'US Virgin Islands': 80, 'Martinique': 81,
           'Madagascar': 82, 'Switzerland': 83, 'Czechia': 84, 'Croatia': 85, 'Jordan': 86, 'Venezuela': 87,
           'Australia': 88, 'Bulgaria': 89, 'Bangladesh': 90, 'San Marino': 91, 'Macao': 92, 'Latvia': 93, 'Peru': 94,
           'United States': 95, 'Argentina': 96, 'Lesotho': 97, 'Malta': 98, 'Poland': 99, 'Hungary': 100,
           'Malaysia': 101, 'Guatemala': 102, 'Montenegro': 103, 'Botswana': 104, 'Netherlands': 105, 'Sri Lanka': 106,
           'Bermuda': 107, 'Belarus': 108, 'Russia': 109, 'Brazil': 110, 'Antarctica': 111, 'Costa Rica': 112,
           'Finland': 113, 'Norway': 114, 'Qatar': 115, 'South Africa': 116, 'Monaco': 117, 'Germany': 118,
           'Albania': 119, 'Uruguay': 120, 'Egypt': 121, 'Northern Mariana Islands': 122,
           'South Georgia and South Sandwich Islands': 123}

In [ ]:
numclasses = len(classes)

In [106]:
print(numclasses)

124


In [89]:
classes_sorted = [x[0] for x in sorted(classes.items(), key=lambda item: item[1])]
print(classes_sorted)

['American Samoa', 'Palestine', 'Sweden', 'Guam', 'Kenya', 'Vietnam', 'Chile', 'Ireland', 'Armenia', 'Canada', 'Ghana', 'Tunisia', 'Bhutan', 'Colombia', 'Philippines', 'Belgium', 'Greece', 'Ukraine', 'Tanzania', 'Kyrgyzstan', 'Slovakia', 'Eswatini', 'Serbia', 'Paraguay', 'Indonesia', 'Spain', 'Isle of Man', 'China', 'North Macedonia', 'Curacao', 'Ecuador', 'Gibraltar', 'Italy', 'Lebanon', 'Pitcairn Islands', 'Puerto Rico', 'Bolivia', 'Nigeria', 'Hong Kong', 'South Sudan', 'Uganda', 'Iraq', 'New Zealand', 'Dominican Republic', 'Estonia', 'Greenland', 'Luxembourg', 'Senegal', 'Mongolia', 'India', 'Pakistan', 'United Arab Emirates', 'Austria', 'Singapore', 'Nepal', 'Cambodia', 'Aland', 'Svalbard and Jan Mayen', 'Lithuania', 'Mozambique', 'Denmark', 'Israel', 'United Kingdom', 'France', 'Thailand', 'Reunion', 'Iceland', 'Taiwan', 'Jersey', 'Andorra', 'Faroe Islands', 'Mexico', 'Turkey', 'Romania', 'Japan', 'Myanmar', 'South Korea', 'Laos', 'Portugal', 'Slovenia', 'US Virgin Islands', 'Mart

In [135]:
def get_country_class(y_pred : [float]):
    max_index = np.argmax(y_pred, axis=0)

    if isinstance(max_index, list):
        if len(max_index) == 0:
            return 0
        else:
            return max_index[0]

    return max_index

def get_country_name(index : int):
    return classes_sorted[index]

In [91]:


def readData(fileNames, dataDir):
    '''
    Takes a list of location folder names and ouputs a list of input image vectors and ouput categorical grid vector pairs.
    fileNames should look like: 60+48.4271513,-110.5611851
    '''
    #     numClasses = self.model.layers[-1].output_shape[-1]
    #     inputShape = self.model.layers[0].input_shape[2:4]
    # load three images in a folder as arrays
    X = np.array(list(map(lambda x: np.array(load_img(dataDir + x)), fileNames)))
    # load grid numbers from folder names and convert them to categorical ouput vectors
    y = tf.keras.utils.to_categorical(list(map(lambda x: classes[x.split("/")[0]], fileNames)),
                                      num_classes=numclasses)
    return X, y


def dataGen(fileNames, dataDir, batchSize=10, infinite=True):
    '''
    Takes a list of location folder names and ouputs
    a list of input image vectors and ouput categorical grid vector pairs in batches.
    The function is essentially used as a generator that calls readData in datches.
    Inifinit: Tells the function to stop or keep going once the list of file names has been iterated through
    fileNames should look like: 60+48.4271513,-110.5611851
    '''
    totalBatches = len(fileNames) / batchSize
    counter = 0
    while (True):
        prev = batchSize * counter
        nxt = batchSize * (counter + 1)
        counter += 1
        yield readData(fileNames[prev:nxt], dataDir)
        if counter >= totalBatches:
            if infinite:
                counter = 0
            else:
                break

In [92]:
it = dataGen(fn1, directory)

In [93]:
X, y = next(it)

In [94]:
outputShape = y.shape
outputShape

(10, 124)

In [95]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [96]:
inputShape = X.shape
inputShape = (1, *inputShape[1:])
inputShape

(1, 662, 1536, 3)

In [97]:
X

array([[[[194, 200, 212],
         [194, 200, 212],
         [194, 201, 211],
         ...,
         [159, 158, 156],
         [159, 158, 156],
         [159, 158, 156]],

        [[192, 198, 210],
         [192, 198, 210],
         [192, 199, 209],
         ...,
         [159, 158, 156],
         [159, 158, 156],
         [159, 158, 156]],

        [[190, 196, 208],
         [191, 197, 209],
         [192, 198, 210],
         ...,
         [159, 158, 156],
         [159, 158, 156],
         [159, 158, 156]],

        ...,

        [[107,  92,  89],
         [107,  92,  89],
         [106,  91,  88],
         ...,
         [ 42,  32,  33],
         [ 43,  33,  34],
         [ 43,  33,  34]],

        [[106,  91,  88],
         [105,  90,  87],
         [104,  89,  86],
         ...,
         [ 35,  25,  26],
         [ 36,  26,  27],
         [ 37,  27,  28]],

        [[105,  90,  87],
         [103,  88,  85],
         [102,  87,  84],
         ...,
         [ 72,  62,  63],
        

In [154]:
from tensorflow.keras.models import load_model

model = load_model('models/model_0_30.h5')

model

In [155]:
y_pred = []
y_true = []

i = 0
for X, y in dataGen(fn1, directory, batchSize=1, infinite=False):
    yp = model.predict(X)
    y_p_cl = get_country_class(yp[0])
    y_pred.append(y_p_cl)

    y_t_cl = get_country_class(y[0])
    y_true.append(get_country_class(y[0]))

    if i == 20:
        break
    else:
        i += 1

1/1 [==============================] - 1s 1s/step


In [156]:
from sklearn.metrics import accuracy_score, f1_score

sett = set()

print(y_pred)

for (y_t, y_p) in list(zip(y_true, y_pred)):

    tup = (get_country_name(y_t), get_country_name(y_p))
    if (y_p == 0):
        tup[1] = None

    if not (tup in sett):
        sett.add(tup)

accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
print("Accuracy:", accuracy)

f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
print("F1 micro:", f1_micro)

f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
print("F1 macro:", f1_macro)


[109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109]
Accuracy: 0.0
F1 micro: 0.0
F1 macro: 0.0
